# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [8]:
!pip install duckdb
!pip install jupysql
!pip install duckdb-engine


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [35]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:/Users/epugnalo/OneDrive - Telefonica/Documents/Data Mining UBA/2C - DM en Economica y Finanzas/datasets/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [12]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [4]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [36]:
%%sql

CREATE or replace table competencia_01_clase as 

SELECT a.*,
    CASE WHEN a.foto_mes<>202103 THEN ''
        WHEN c.numero_de_cliente IS NOT null THEN 'CONTINUA'    --cliente esta en Mes 05 : continua
        WHEN b.numero_de_cliente is null THEN 'BAJA+1'           --cliente no esta en Mes 04: baja+1
        ELSE 'BAJA+2' END clase_ternaria                         --cliente no esta en Mes 05: baja+2
FROM competencia_01 a
LEFT JOIN competencia_01 b ON b.foto_mes=202104 AND a.numero_de_cliente = b.numero_de_cliente
LEFT JOIN competencia_01 c ON c.foto_mes=202105 AND a.numero_de_cliente = c.numero_de_cliente

;

SELECT foto_mes, clase_ternaria, count(distinct numero_de_cliente) cant_clientes
FROM competencia_01_clase
GROUP BY  foto_mes, clase_ternaria
;


,foto_mes,clase_ternaria,cant_clientes
0,202104,,163637
1,202103,CONTINUA,161312
2,202105,,164102
3,202103,BAJA+1,1049
4,202103,BAJA+2,963


## Agregando Columnas

In [15]:
# select
#     Master_Fvencimiento
#     , Visa_Fvencimiento
#     , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
#     , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
# from competencia_01

# select
#     Master_msaldototal
#     , Visa_msaldototal
#     , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
# from competencia_01

# CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);


# select distinct
#     Master_msaldototal
#     , Visa_msaldototal
#     , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
# from competencia_01


In [4]:
%%sql

CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);
CREATE MACRO resta_sin_null(a, b) AS ifnull(a, 0) - ifnull(b, 0);
CREATE MACRO DivisionPorCero(a, b) AS  CASE WHEN b = 0 THEN null ELSE ifnull(a,0) / b  END;


,Success


In [37]:
%%sql

CREATE or replace table competencia_01_FE_i1 as 

select *,

    -- VISA + MASTERCARD  (MAXIMOS Y MINIMOS)

    greatest(Master_delinquency, Visa_delinquency) as tc_delinquency,
    greatest(Master_status, Visa_status) as tc_status_mayor,
    least(Master_status, Visa_status) as tc_status_menor,
    greatest(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as tc_mfinanciacion_mayor,
    least(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as tc_mfinanciacion_menor,    
    greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor,
    least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor,  
    greatest(Master_Finiciomora, Visa_Finiciomora) as tc_Finiciomora_mayor ,	
    least(Master_Finiciomora, Visa_Finiciomora) as tc_Finiciomora_menor ,
    greatest(Master_msaldototal, Visa_msaldototal) as tc_msaldototal_mayor ,	
    least(Master_msaldototal, Visa_msaldototal) as tc_msaldototal_menor ,
    greatest(Master_msaldopesos, Visa_msaldopesos) as tc_msaldopesos_mayor ,	
    least(Master_msaldopesos, Visa_msaldopesos) as tc_msaldopesos_menor ,
    greatest(Master_msaldodolares, Visa_msaldodolares) as tc_msaldodolares_mayor ,	
    least(Master_msaldodolares, Visa_msaldodolares) as tc_msaldodolares_menor ,
    greatest(Master_mconsumospesos, Visa_mconsumospesos) as tc_mconsumospesos_mayor ,	
    least(Master_mconsumospesos, Visa_mconsumospesos) as tc_mconsumospesos_menor ,
    greatest(Master_mconsumosdolares, Visa_mconsumosdolares) as tc_mconsumosdolares_mayor ,	
    least(Master_mconsumosdolares, Visa_mconsumosdolares) as tc_mconsumosdolares_menor ,
    greatest(Master_mlimitecompra, Visa_mlimitecompra) as tc_mlimitecompra_mayor ,	
    least(Master_mlimitecompra, Visa_mlimitecompra) as tc_mlimitecompra_menor ,
    greatest(Master_madelantopesos, Visa_madelantopesos) as tc_madelantopesos_mayor ,	
    least(Master_madelantopesos, Visa_madelantopesos) as tc_madelantopesos_menor ,
    greatest(Master_madelantodolares, Visa_madelantodolares) as tc_madelantodolares_mayor ,	
    least(Master_madelantodolares, Visa_madelantodolares) as tc_madelantodolares_menor ,
    greatest(Master_fultimo_cierre, Visa_fultimo_cierre) as tc_fultimo_cierre_mayor ,	
    least(Master_fultimo_cierre, Visa_fultimo_cierre) as tc_fultimo_cierre_menor ,
    greatest(Master_mpagado, Visa_mpagado) as tc_mpagado_mayor ,	
    least(Master_mpagado, Visa_mpagado) as tc_mpagado_menor ,
    greatest(Master_mpagospesos, Visa_mpagospesos) as tc_mpagospesos_mayor ,	
    least(Master_mpagospesos, Visa_mpagospesos) as tc_mpagospesos_menor ,
    greatest(Master_mpagosdolares, Visa_mpagosdolares) as tc_mpagosdolares_mayor ,	
    least(Master_mpagosdolares, Visa_mpagosdolares) as tc_mpagosdolares_menor ,
    greatest(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_mayor ,	
    least(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_menor ,
    greatest(Master_mconsumototal, Visa_mconsumototal) as tc_mconsumototal_mayor ,	
    least(Master_mconsumototal, Visa_mconsumototal) as tc_mconsumototal_menor ,
    greatest(Master_cconsumos, Visa_cconsumos) as tc_cconsumos_mayor ,	
    least(Master_cconsumos, Visa_cconsumos) as tc_cconsumos_menor ,
    greatest(Master_cadelantosefectivo, Visa_cadelantosefectivo) as tc_cadelantosefectivo_mayor ,	
    least(Master_cadelantosefectivo, Visa_cadelantosefectivo) as tc_cadelantosefectivo_menor ,
    greatest(Master_mpagominimo, Visa_mpagominimo) as tc_mpagominimo_mayor ,	
    least(Master_mpagominimo, Visa_mpagominimo) as tc_mpagominimo_menor,   

    -- VISA + MASTERCARD (SUMAS)
    suma_sin_null(Visa_mfinanciacion_limite, Master_mfinanciacion_limite) AS tc_mfinanciacion_limite_suma,
    suma_sin_null(Visa_msaldototal, Master_msaldototal) AS tc_msaldototal_suma,
    suma_sin_null(Visa_msaldopesos, Master_msaldopesos) AS tc_msaldopesos_suma,
    suma_sin_null(Visa_msaldodolares, Master_msaldodolares) AS tc_msaldodolares_suma,
    suma_sin_null(Visa_mconsumospesos, Master_mconsumospesos) AS tc_mconsumospesos_suma,
    suma_sin_null(Visa_mconsumosdolares, Master_mconsumosdolares) AS tc_mconsumosdolares_suma,
    suma_sin_null(Visa_mlimitecompra, Master_mlimitecompra) AS tc_mlimitecompra_suma,
    suma_sin_null(Visa_madelantopesos, Master_madelantopesos) AS tc_madelantopesos_suma,
    suma_sin_null(Visa_madelantodolares, Master_madelantodolares) AS tc_madelantodolares_suma,
    suma_sin_null(Visa_mpagado, Master_mpagado) AS tc_mpagado_suma,
    suma_sin_null(Visa_mpagospesos, Master_mpagospesos) AS tc_mpagospesos_suma,
    suma_sin_null(Visa_mpagosdolares, Master_mpagosdolares) AS tc_mpagosdolares_suma,
    suma_sin_null(Visa_mconsumototal, Master_mconsumototal) AS tc_mconsumototal_suma,
    suma_sin_null(Visa_cconsumos, Master_cconsumos) AS tc_cconsumos_suma,
    suma_sin_null(Visa_cadelantosefectivo, Master_cadelantosefectivo) AS tc_cadelantosefectivo_suma,
    suma_sin_null(Visa_mpagominimo, Master_mpagominimo) AS tc_mpagominimo_suma,


    -- OTRAS SUMAS y RESTAS
    suma_sin_null(mpasivos_margen, mactivos_margen) AS m_margen_suma,
    suma_sin_null(ccuenta_corriente, ccaja_ahorro) AS c_ccorrientes_cahorro_suma,
    suma_sin_null(mcuenta_corriente, mcuenta_corriente_adicional) AS m_ccorrientes_suma,
    suma_sin_null(mcaja_ahorro, mcaja_ahorro_adicional) AS m_cahorro_suma,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) AS c_tarjetas_suma,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS  c_tarjetas_transacciones_suma,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) AS  mtarjetas_consumo_suma,

    suma_sin_null(cprestamos_prendarios, suma_sin_null(cprestamos_personales, cprestamos_hipotecarios)) AS  cprestamos_suma,
    suma_sin_null(mprestamos_personales, suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios)) AS  mprestamos_suma,

    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_dolares) AS mplazo_fijo_suma,
    suma_sin_null(minversion1_pesos, suma_sin_null(minversion1_dolares, minversion2)) AS  minversion_suma,
    suma_sin_null(cseguro_vida,suma_sin_null(cseguro_auto, suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales))) AS  cseguro_suma,

    suma_sin_null(mpayroll, mpayroll2) AS mpayroll_suma,
    suma_sin_null(ccuenta_debitos_automaticos, suma_sin_null(ctarjeta_visa_debitos_automaticos, ctarjeta_master_debitos_automaticos)) AS  c_debitos_automaticos_suma,
    suma_sin_null(mcuenta_debitos_automaticos, suma_sin_null(mttarjeta_visa_debitos_automaticos, mttarjeta_master_debitos_automaticos)) AS  m_debitos_automaticos_suma,

    suma_sin_null(mcajeros_propios_descuentos, suma_sin_null(mtarjeta_visa_descuentos, mtarjeta_master_descuentos)) AS  m_descuentos_suma,
    suma_sin_null(mcomisiones_mantenimiento, mcomisiones_otras) AS mcomisiones_suma,
   
    suma_sin_null(ctransferencias_recibidas, ctransferencias_emitidas) AS ctransferencias_suma,
    resta_sin_null(ctransferencias_recibidas, ctransferencias_emitidas) AS ctransferencias_resta,
    resta_sin_null(mtransferencias_recibidas, mtransferencias_emitidas) AS mtransferencias_resta,

    suma_sin_null(mtransferencias_emitidas, mextraccion_autoservicio) AS m_extracciones_transf_suma,
    mpayroll+ mpayroll2+mcajeros_propios_descuentos+suma_sin_null(mtarjeta_visa_descuentos,mtarjeta_master_descuentos)+mtransferencias_recibidas+mcheques_depositados AS m_movimientos_positivos_suma,
    mautoservicio+mtarjeta_visa_consumo+mtarjeta_master_consumo+mcuenta_debitos_automaticos+mttarjeta_visa_debitos_automaticos+mttarjeta_master_debitos_automaticos+mpagodeservicios+mpagomiscuentas+mcomisiones_mantenimiento+mcomisiones_otras+mtransferencias_emitidas+mextraccion_autoservicio+mcheques_emitidos AS m_movimientos_negativos_suma

 from competencia_01_clase



,Success


ctrx_quarter(mov de voluntarios ult 90 dias (no tarketa))
mcuentas_saldo
mactivos_margen
mprestamos_personales
mcuenta_corriente
mcaja_ahorro
mcomisiones
ctarjeta_visa
ctarjeta_master
ccomisiones_otras

----
comisiones vs saldo (o otro)
prestamos vs saldo (o otros)
descuentos vs saldo (o otro)


In [39]:
%%sql

CREATE or replace table competencia_01_FE_i2 as

select *,
    DivisionPorCero(Visa_mconsumototal, Visa_mlimitecompra)*100 Visa_consumo_div_limite,
    DivisionPorCero(Master_mconsumototal, Master_mlimitecompra)*100 Master_consumo_div_limite,
    DivisionPorCero(tc_mconsumototal_suma, tc_mlimitecompra_suma)*100 tc_consumo_div_limite,
    DivisionPorCero(tc_mconsumototal_suma, tc_msaldototal_suma)*100 tc_consumo_div_saldo,
    DivisionPorCero(tc_mpagado_suma, tc_msaldototal_suma)*100 tc_pagado_div_saldo,
    DivisionPorCero(tc_mpagado_suma, tc_msaldototal_suma)*100 tc_pagado_div_saldo,    

    DivisionPorCero(mprestamos_suma, mcuentas_saldo)*100 prestamos_saldo_div,    
    DivisionPorCero(m_descuentos_suma, mcuentas_saldo)*100 descuentos_saldo_div,    
    DivisionPorCero(mcomisiones_suma, mcuentas_saldo)*100 comisiones_saldo_div, 

    DivisionPorCero(m_extracciones_transf_suma, mcuentas_saldo)*100 extracciones_saldo_div, 
    DivisionPorCero(m_movimientos_negativos_suma, mcuentas_saldo)*100 mnegativos_saldo_div, 
    DivisionPorCero(m_movimientos_negativos_suma, m_movimientos_positivos_suma)*100 mpositivos_negativos_div

from competencia_01_FE_i1
;

,Success


In [40]:
%%sql

CREATE or replace table competencia_01_FE_i3 as

select *,
    percent_rank() over (partition by foto_mes order by m_movimientos_negativos_suma) as m_movimientos_negativos_suma_perc,
    percent_rank() over (partition by foto_mes order by mprestamos_suma) as mprestamos_suma_perc,
    percent_rank() over (partition by foto_mes order by prestamos_saldo_div) as prestamos_saldo_div_perc,
    percent_rank() over (partition by foto_mes order by mcuentas_saldo) as mcuentas_saldo_perc,
    percent_rank() over (partition by foto_mes order by m_ccorrientes_suma) as m_ccorrientes_suma_perc,
    percent_rank() over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_perc,
    percent_rank() over (partition by foto_mes order by tc_mfinanciacion_limite_suma) as tc_mfinanciacion_limite_suma_perc,   

    percent_rank() over (partition by foto_mes order by tc_mpagominimo_suma) as tc_mpagominimo_suma_perc,
    percent_rank() over (partition by foto_mes order by tc_mpagominimo_mayor) as tc_mpagominimo_mayor_perc,
    percent_rank() over (partition by foto_mes order by Visa_msaldototal) as Visa_msaldototal_perc,
    percent_rank() over (partition by foto_mes order by mplazo_fijo_dolares) as mplazo_fijo_dolares_perc

from competencia_01_FE_i2


,Success


In [47]:
%%sql

select m_movimientos_negativos_suma
from competencia_01_FE_i3

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.BinderException) Binder Error: Referenced column "m_movimientos_negativos_suma" not found in FROM clause!
Candidate bindings: "competencia_01_FE_i3.m_movimientos_negativos_suma_perc"
LINE 1: select m_movimientos_negativos_suma
               ^
[SQL: select m_movimientos_negativos_suma
from competencia_01_FE_i3]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [48]:
%%sql

ALTER TABLE competencia_01_FE_i3
DROP COLUMN m_movimientos_negativos_suma;

ALTER TABLE competencia_01_FE_i3
DROP COLUMN mprestamos_suma
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN prestamos_saldo_div
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN mcuentas_saldo
;

ALTER TABLE competencia_01_FE_i3
DROP COLUMN m_ccorrientes_suma
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN mcuenta_corriente
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN tc_mfinanciacion_limite_suma
;

ALTER TABLE competencia_01_FE_i3
DROP COLUMN tc_mpagominimo_suma
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN tc_mpagominimo_mayor
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN Visa_msaldototal
;
ALTER TABLE competencia_01_FE_i3
DROP COLUMN mplazo_fijo_dolares
;


,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [13]:
%%sql

SELECT foto_mes, clase_ternaria, count(distinct numero_de_cliente) cant_clientes
FROM competencia_01_clase
GROUP BY  foto_mes, clase_ternaria
;

SELECT foto_mes, AVG(CASE WHEN clase_ternaria='BAJA+2' THEN 1 ELSE 0 END)*100 prop_bajas_menos_2
FROM competencia_01_clase
GROUP BY foto_mes
;


,foto_mes,prop_bajas_menos_2
0,202103,0.589626
1,202104,0.000000
2,202105,0.000000


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [52]:
%%sql
COPY competencia_01_FE_i3 TO 'C:/Users/epugnalo/OneDrive - Telefonica/Documents/Data Mining UBA/2C - DM en Economica y Finanzas/datasets/competencia_01_FE_v02.csv' (FORMAT CSV, HEADER)


,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [14]:
%%sql
create or replace table ejercicio_target as 
select 
    *
from read_csv_auto('C:/Users/emiba/Documents/DMenEyF/ejercicio_target.csv')


,Success


In [15]:
%%sql
select * from ejercicio_target

,periodo,nombre
0,202110,Isaac Valverde
1,202110,Santiago Córdova Tercero
2,202110,Jerónimo Ocasio
3,202110,Isabel Terrazas Hijo
4,202110,Diego Caballero
...,...,...
734,202203,Srita. Violeta Carrero Segundo
735,202203,Valentín Delafuente
736,202203,Valeria Candelaria
737,202203,Vicente Luna Tercero


In [52]:
%%sql
select periodo, nombre
from ejercicio_target
group by periodo, nombre
having count(*)>2

-- No hay dupluicados

,periodo,nombre


In [ ]:

WITH 